In [2]:
import numpy as np
import pandas as pd
import cv2
import os
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization, Activation, GlobalAveragePooling2D
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, DenseNet121
from sklearn.model_selection import StratifiedKFold


import tensorflow_hub as hub
import tensorflow as tf
import sklearn

NUMBER_CLASSES = 4

In [3]:
def create_vgg16_model(image_size):
    base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3), pooling=None)
    base_model.trainable = False
    
    model = Sequential()
    model.add(base_model)
    model.add(Conv2D(2048, (1, 1), activation='relu')) # Transition Layer
    model.add(GlobalAveragePooling2D())   
    model.add(Dense(NUMBER_CLASSES, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=[AUC(multi_label=True)])
    return model

In [4]:
def normalize_image(img, label):
    img = tf.cast(img, np.float32)# / 255.0
    img = tf.keras.applications.resnet.preprocess_input(img)
    return img, label

def create_train_dataset(images_train, labels_train, batch_size=16):
    train_dataset = tf.data.Dataset.from_tensor_slices((images_train, labels_train))
    train_dataset = train_dataset.map(normalize_image)
    train_dataset = train_dataset.shuffle(buffer_size=100)
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
    train_dataset = train_dataset.repeat()
    return train_dataset

def create_val_dataset(images_val, labels_val, batch_size=16):
    val_dataset = tf.data.Dataset.from_tensor_slices((images_val, labels_val))
    val_dataset = val_dataset.map(normalize_image)
    val_dataset = val_dataset.batch(batch_size, drop_remainder=False)
    return val_dataset

In [5]:
def evaluate_model(model, images_test, labels_test, batch_size=16):

    images_test_2 = tf.cast(images_test, np.float32) / 255.0
    predictions = model.predict(images_test_2, batch_size=batch_size)
    
    auc_per_class = []
    
    for class_idx in range(NUMBER_CLASSES):
        true_labels_class = labels_test[:, class_idx]
        
        auc = sklearn.metrics.roc_auc_score(true_labels_class, predictions[:, class_idx])
        auc_per_class.append(auc)
        
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels_test, axis=1)
    balanced_accuracy = sklearn.metrics.balanced_accuracy_score(true_classes, predicted_classes)
    accuracy = sklearn.metrics.accuracy_score(true_classes, predicted_classes)
    
    return auc_per_class, balanced_accuracy, accuracy

In [15]:
BATCH_SIZE = 16
NUMBER_OF_IMAGES = 20000
IMAGE_SIZE = 224
evaluations={}

with open(f"augmentation_data/images_clean.pkl", "rb") as file:
    images_clean = pickle.load(file)[:100]
with open(f"augmentation_data/labels_clean.pkl", "rb") as file:
    labels_clean = pickle.load(file)[:100]
with open(f"augmentation_data/images_test.pkl", "rb") as file:
    images_test = pickle.load(file)[:100]
with open(f"augmentation_data/labels_test.pkl", "rb") as file:
    labels_test = pickle.load(file)[:100]

In [ ]:
with open(f"augmentation_data/images_horizontal_flip.pkl", "rb") as file:
    images_horizontal_flip = pickle.load(file)
with open(f"augmentation_data/images_horizontal_flip.pkl", "rb") as file:
    images_vertical_flip = pickle.load(file)
with open(f"augmentation_data/images_rotation.pkl", "rb") as file:
    images_rotation = pickle.load(file)
with open(f"augmentation_data/images_crop.pkl", "rb") as file:
    images_crop = pickle.load(file)

In [16]:
with open(f"augmentation_data/images_test.pkl", "rb") as file:
    images_test = pickle.load(file)
with open(f"augmentation_data/labels_test.pkl", "rb") as file:
    labels_test = pickle.load(file)


for aug_idx in range(0, 5):
    
    images
    
    if aug_idx == 0:
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


    for run, (train_idx, val_idx) in enumerate(skf.split(images_clean, np.argmax(labels_clean, axis=1))):
    
        tf.keras.backend.clear_session()
                
        images_train, images_val = images_clean[train_idx], images_clean[val_idx]
        labels_train, labels_val = labels_clean[train_idx], labels_clean[val_idx]
           
        train_dataset = create_train_dataset(images_train, labels_train)
        val_dataset = create_val_dataset(images_val, labels_val)
        
        steps_per_epoch = len(images_train) // BATCH_SIZE
        validation_steps = len(images_val) // BATCH_SIZE
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        reduceLR_callback = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=3,
            min_lr=1e-6
        )     
        
        model = create_mobilenetv2_model(IMAGE_SIZE)
                    
        model.fit(
            train_dataset,
            steps_per_epoch=steps_per_epoch,
            epochs=50,
            validation_data=val_dataset,
            validation_steps=validation_steps,
            batch_size=BATCH_SIZE,
            verbose=1,
            callbacks=[early_stopping, reduceLR_callback]) #######BIRNE
        
        sub_eval = evaluate_model(model, images_test, labels_test)
        
        evaluations[run] = sub_eval
        print(evaluations)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 652ms/step - auc: 0.3901 - loss: 1.4385 - val_auc: 0.5087 - val_loss: 1.4302 - learning_rate: 0.0010
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 284ms/step - auc: 0.5557 - loss: 1.2329 - val_auc: 0.6582 - val_loss: 1.1895 - learning_rate: 0.0010
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 270ms/step - auc: 0.7113 - loss: 1.1164 - val_auc: 0.5987 - val_loss: 1.3409 - learning_rate: 0.0010
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step - auc: 0.6274 - loss: 1.1360 - val_auc: 0.6322 - val_loss: 1.2286 - learning_rate: 0.0010
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 286ms/step - auc: 0.7241 - loss: 1.0832 - val_auc: 0.6683 - val_loss: 1.1621 - learning_rate: 0.0010
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step - auc: 0.7663 - loss: 1.0454 - val_auc: 0.6665 - val_loss: 1.1778 - learning_rate: 0.0010
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step - auc: 0.7981 - loss: 0.9937 - val_auc: 0.6814 - val_loss: 1.1751 - learning_rate: 0.0010
Epoch 8/50
5/5 ━━━━━